## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [6]:
import matplotlib.pyplot as plt
# import activity_classifier_utils
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import scipy.signal

In [2]:
# HYPERPARAMETER

high_pass = 40/ 60 # changing BPM to Hz
low_pass = 240/60 # changing BPM to Hz

fs = 125



In [3]:
import glob

import numpy as np
import scipy as sp
import scipy.io


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl ,high_pass = 40/ 60,low_pass = 240/60,fs = 125):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # loading the reference file
    ground_truth = sp.io.loadmat(ref_fl)['BPM0']
    
    # bandpass filtering the signals
    accx = BandpassFilter(accx, (high_pass,low_pass), fs=fs)
    accy = BandpassFilter(accy, (high_pass,low_pass), fs=fs)
    accz = BandpassFilter(accz, (high_pass,low_pass), fs=fs)

    acc = acc_mag(accx,accy,accz) # calculatnig the magnitude of the accelerometers
    
    ppg =  BandpassFilter(ppg, (high_pass,low_pass), fs=fs)
    
    confidence, estimates = get_estimates(ppg, acc, fs)
    
    estimates = estimates*60
    
    
    # Compute pulse rate estimates and estimation confidence.
#     print(ground_truth.shape, estimates.shape, ppg.shape, confidence.shape)
#     print(data_fl)
#     print(ref_fl)
    errors = absolute_error(ground_truth, estimates)
    
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.

    return errors, confidence

In [4]:
def absolute_error(y_true, y_pred):
    
    return np.abs(np.diag(np.subtract(y_true,y_pred)))

def acc_mag(accx,accy,accz):
    """
    calculates the magnitude of the the accelerometers 
    
    """
    return np.sqrt(accx**2 + (accy - np.mean(accy))**2 + accz**2)

def BandpassFilter(signal, band_pass = (40/60, 240/60), fs = fs):
    
    """
    performs a bandpass filter on the provided data.
    """
    
    b,a = scipy.signal.butter(3, band_pass, fs=fs, btype= 'bandpass')
    return scipy.signal.filtfilt(b,a,signal)

def getpeaks(signal,fs, height = None, width = None ):
    
    """
    performs fourier transfom then finds the peaks in the frequency domain.
    
    returns the peaks values, corresponding frequencies and the fourier trasform
    """

    freqs= np.fft.rfftfreq(2*len(signal), 1/fs)

    fourier= np.abs(np.fft.rfft(signal, 2*len(signal)))

    pks = sp.signal.find_peaks(fourier, height = height, distance = width)[0]
    
    piks = []
    herzs = []

    for pik in pks:
        piks.append(fourier[pik])
    piks = sorted(piks)
    
    for pik in piks:
        herzs.append(freqs[np.where(fourier == pik)[0][0]])
        
    piks = piks[::-1]
    herzs = herzs[::-1]

    return piks, herzs, fourier


def get_estimates(ppg, acc, fs, ppg_height = None, ppg_width = 10, acc_height = None, acc_width = 10):
    
    """
    calcultates the heart rate estimation and confidence
    """
    
    pred_bpm = []
    conf = []

    
    
    for i in range( int(((ppg.shape[0]/fs)-8)/2)+1 ):

        ppg_window = ppg[i*2*fs:(8*fs+i*2*fs)]
        acc_window = acc[i*2*fs:(8*fs+i*2*fs)]
        ppg_pks,ppg_herz, ppg_f = getpeaks(ppg_window, fs,height= ppg_height, width= ppg_width)
        P,acc_herz, F = getpeaks(acc_window, fs,height= acc_height, width= acc_width)


        for i in range(len(ppg_herz)):
            if ppg_herz[i] not in acc_herz:
                pred_bpm.append(ppg_herz[i])
                conf.append(ppg_pks[i]/ppg_f.sum())
                break

    conf = np.array(conf)
    pred_bpm = np.array(pred_bpm)
    
    return conf, pred_bpm

In [5]:
Evaluate()

30.158367443457731

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...


> - **Code Description** - 
    In order to run the code for all the subjects, you only need to run Evaluate().
    But if you want to run it for a single subjet you need to do the following:
    - Load the data witht the given functions for the TROIKA data and refrence files.
    - run the bandpassfilter function on both the ppg and accx,accy and accz data.
    - run the acc_mag function on the three accelerator data.
    - runt the get_estimates function with the filtered ppg and the magnitude acc signals, this function will return the estimates and condifence.

> - **Data Description** - The dataset consisted of ppg signals as well as accelerometers in three directions. the data even after accounting for the movement of the (using the acceleroter signals) is still noisy. maybe a second ppg sensor at another location would help making a better dataset along with labels for the activities.

> - **Algorithhm Description** will include the following:
>   - The algorithm detects the peaks in the ppg and acc signals in the frequency domain, then compare them inorder to exclude the frequencies caused by motion artifact,and chooses the strongest frequency in the ppg signal that is not in the acc frequencies.

>   - it takes advantage if the blood flow movement caused by the heart rate, where more heart rate is transferred to be more blood movement which what the algorithm looks at as frequencies.

>   - The algorithm outputs the heart rate estimation and the confidence for a total of 8 seconds, and outputs every 2 seconds afterwards.

>   - the algorithm can be affected by sources of noises and other noise signals 

>   - The algorith doesnt perform very well in stages of changing heart rate

> - **Algorithm Performance** - the metric for the algorithm performance is mean absolute error.


-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 